In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

base_path = '/content/drive/MyDrive/dataset'
train_dir = os.path.join(base_path, 'train')
test_dir = os.path.join(base_path, 'test')

def load_images(directory):
    images = []
    labels = []
    classes = ['covid', 'normal']

    for class_idx, class_name in enumerate(classes):
        class_path = os.path.join(directory, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(class_idx)

    return np.array(images), np.array(labels)

X_train, y_train = load_images(train_dir)
X_test, y_test = load_images(test_dir)

def extract_gabor_features(images):
    features = []
    ksize = 31
    sigma = 4.0
    lambd_values = [10.0, 20.0]
    gamma = 0.5
    psi = 0
    thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]

    kernels = []
    for lambd in lambd_values:
        for theta in thetas:
            kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
            kernels.append(kernel)

    for img in images:
        feats = []
        for kernel in kernels:
            filtered = cv2.filter2D(img, cv2.CV_32F, kernel)
            feats.append(filtered.mean())
            feats.append(filtered.var())
        features.append(feats)
    return np.array(features)

X_train_gabor = extract_gabor_features(X_train)
X_test_gabor = extract_gabor_features(X_test)

svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True)
svm.fit(X_train_gabor, y_train)

y_pred = svm.predict(X_test_gabor)

print("Gabor-based Approach Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred))

plt.figure(figsize=(6,4))
plt.imshow(confusion_matrix(y_test, y_pred), cmap='Blues')
plt.title('Gabor+SVM Confusion Matrix')
plt.xticks([0,1], ['covid', 'normal'])
plt.yticks([0,1], ['covid', 'normal'])
plt.colorbar()
plt.show()

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



ERROR: Could not find a version that satisfies the requirement google-colab (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for google-colab


ModuleNotFoundError: No module named 'google.colab'